## Problem Statement:

Solve:
$$f = -\Delta{u} + \nabla{p} $$
when:
$$ u = \nabla \times sin(\pi x y) \ and \ p = sin(2\pi x) $$
where $\nabla \times$ is defined as $(-\frac{\partial}{\partial{y}}, \frac{\partial}{\partial{x}})$

to get f we derive from the Stokes probem given the method of manufactured solutions:
$$ f = \Delta(\nabla \times sin(\pi x y)) + \nabla{sin(2\pi x)} $$

same for boundary conditions.

In [ ]:
from dolfin import *
import matplotlib.pyplot as plt
import numpy as np
fig_size = (24, 24)
%matplotlib inline

plt.rcParams["figure.figsize"] = fig_size

In [ ]:
def stokes_solver(h, u_deg, p_deg):

    mesh = UnitSquareMesh(h, h)
    
    def right(x, on_boundary): return near(x[0], 1.)
    
    sub_domains = FacetFunction('size_t', mesh, 0)
    DomainBoundary().mark(sub_domains, 1)
    
    outflow = AutoSubDomain(right)
    outflow.mark(sub_domains, 2)
    #p_e = Expression("sin(2*pi*x[0])", degree=4, domain=mesh)
    #u_e_ = Expression("sin(pi*x[0]*x[1])", degree=4, domain=mesh)
    x = SpatialCoordinate(mesh)
    p_e = sin(2*pi*x[0])
    u_e = sin(pi*x[0]*x[1])
    u_e = curl(u_e) 
    f = -div(grad(u_e)) + grad(p_e)

    ds_ = Measure('ds', domain=mesh, subdomain_data=sub_domains)

    n = FacetNormal ( mesh )
    # Define function spaces
    P2 = VectorElement("Lagrange", mesh.ufl_cell(), u_deg)
    P1 = FiniteElement("Lagrange", mesh.ufl_cell(), p_deg)
    TH = P2 * P1
    W = FunctionSpace(mesh, TH)

    u_projected = project(u_e, FunctionSpace(mesh, VectorElement("Lagrange", mesh.ufl_cell(), u_deg+3)))
    p_projected = project(p_e, FunctionSpace(mesh, FiniteElement("Lagrange", mesh.ufl_cell(), p_deg+3)))
    u_bc = DirichletBC(W.sub(0), u_projected, sub_domains, 1)
    #p_bc = DirichletBC(W.sub(1), p_projected, "on_boundary")


    bcs = [u_bc]

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    #f = Constant((0, 0))
    a = (inner(grad(u), grad(v)) - div(v)*p + q*div(u))*dx 
    
    n = Constant((1, 0))
    g = p_e*n + grad(u_e)*n
    L = inner(f, v)*dx + inner(g, v)*ds_(2)

    # Compute solution
    w = Function(W)
    solve(a == L, w, bcs)

    # Split the mixed solution using deepcopy
    # (needed for further computation on coefficient vector)
    (u, p) = w.split(True)
    
    E_u = errornorm(u_projected, u, norm_type='H1')
    E_p = errornorm(p_projected, p, norm_type='L2')
    return u, p, mesh, E_u, E_p

## Compute the solutions and errors

In [ ]:
hs = [5, 10, 40, 80]

errors = []
for u_deg, p_deg in ((2, 1), (1, 1)):
    print u_deg, p_deg
    Es = []
    for kk, h in enumerate(hs):
        u, p, mesh, E_u, E_p = stokes_solver(h, u_deg, p_deg)
        if Es:
            h = float(h)
            E_u0, E_p0 = Es[-1]
            rate_u = ln(E_u/E_u0)/ln(h/hs[kk-1])
            rate_p = ln(E_p/E_p0)/ln(h/hs[kk-1])
        else:
            rate_u, rate_p = np.nan, np.nan
        print E_u, '@', rate_u, E_p, '@', rate_p
        Es.append((E_u, E_p))
    errors.append(Es)

errors = np.array(errors)

## Plot Errors

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(24, 24))
ax = ax.flatten()
for i, j, k  in zip(range(4), [0, 0, 1,1], [0,1,0,1]):
    if i == 0:
        ax[i].set_ylabel(r"$P_2-P_1 \ Elements \ Error$", fontsize=28)
        ax[i].set_title(r"$H^1 \ Norm \ - \ Velocity$", fontsize=28)
    elif i == 1:
        ax[i].set_title(r"$L_2 \ Norm \ - \ Pressure$", fontsize=28)
    elif i == 2:
        ax[i].set_ylabel(r"$P_1-P_1 \ Elements \ Error$", fontsize=28)
        ax[i].set_xlabel(r"$Meshsize \ h$", fontsize=28)
    elif i == 3:
        ax[i].set_xlabel(r"$Meshsize \ h$", fontsize=28)
    a, b, = np.polyfit(np.log(hs), np.log(errors[j, :, k]), deg=1)
    #print a, b
    ax[i].loglog(hs, errors[j, :, k], c='black', linewidth=4)
    ax[i].grid(which='both')
    slope_str = "Slope: %.1f" % a
    ax[i].annotate(slope_str, xycoords='axes fraction', xy=(0.6, 0.6), xytext=(0.7, 0.6), textcoords='axes fraction', fontsize=32)
    
    for tick in ax[i].xaxis.get_major_ticks():
        tick.label.set_fontsize(24)
    
    for tick in ax[i].yaxis.get_major_ticks():
        tick.label.set_fontsize(24)
        
fig.savefig("stokes_exercise.png", bbox_extra_artists=None, bbox_inches='tight',dpi=72)